In [8]:
import json
import pandas as pd
import pickle
import os
import shutil

In [31]:
'config_msr_action3d_temporal_1024_16_128_128_0.001'.split('_')

['config', 'msr', 'action3d', 'temporal', '1024', '16', '128', '128', '0.001']

In [11]:
hold_out_score = json.load(open(r'holdout_scores.json'))
df_hold_out_score = pd.DataFrame(hold_out_score)
df_hold_out_score

,model,accuracy
0,config_msr_action3d_temporal_1024_16_128_128_0...,4.807692
1,config_msr_action3d_temporal_1024_16_128_128_0...,6.129808
2,config_msr_action3d_temporal_1024_16_128_128_0.05,4.807692
3,config_msr_action3d_temporal_1024_16_128_32_0.001,6.370192
4,config_msr_action3d_temporal_1024_16_128_128_0...,4.447115
...,...,...
118,config_msr_action3d_temporal_512_16_128_32_0.025,4.807692
119,config_msr_action3d_temporal_512_16_128_32_0.05,4.326923
120,config_msr_action3d_temporal_512_16_128_64_0.001,4.206731
121,config_msr_action3d_temporal_512_16_128_64_0.025,4.807692


In [18]:
df_hold_out_score['model'].apply(lambda x: x.split('_')[4] if len(x.split('_')) > 4 else None)

0      1024
1      1024
2      1024
3      1024
4      1024
       ... 
118     512
119     512
120     512
121     512
122    None
Name: model, Length: 123, dtype: object

In [19]:
df_hold_out_score['dim_hidden'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[4] if len(x.split('_')) > 4 else None)
df_hold_out_score['num_heads'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[5] if len(x.split('_')) > 5 else None)
df_hold_out_score['num_inds'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[6] if len(x.split('_')) > 6 else None)
df_hold_out_score['batch_size'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[7] if len(x.split('_')) > 7 else None)
df_hold_out_score['lr'] = df_hold_out_score['model'].apply(lambda x: x.split('_')[8] if len(x.split('_')) > 8 else None)

In [21]:
df_hold_out_score[df_hold_out_score['accuracy'] > 7]

,model,accuracy,dim_hidden,num_heads,num_inds,batch_size,lr
25,config_msr_action3d_temporal_1024_16_64_32_0.001,9.615385,1024,16,64,32,0.001
35,config_msr_action3d_temporal_1024_4_128_32_0.025,9.014423,1024,4,128,32,0.025
78,config_msr_action3d_temporal_2048_16_64_128_0.001,7.091346,2048,16,64,128,0.001
102,config_msr_action3d_temporal_2048_4_32_64_0.001,7.331731,2048,4,32,64,0.001
122,config_msr_action3d_dev-mathew-2,9.765625,None,None,None,None,None


In [36]:
keep_models = df_hold_out_score[df_hold_out_score['accuracy'] > 5]['model'].to_list()
keep_models

['config_msr_action3d_temporal_1024_16_128_128_0.025',
 'config_msr_action3d_temporal_1024_16_128_32_0.001',
 'config_msr_action3d_temporal_1024_16_128_32_0.001',
 'config_msr_action3d_temporal_1024_16_128_64_0.001',
 'config_msr_action3d_temporal_1024_16_128_64_0.025',
 'config_msr_action3d_temporal_1024_16_32_128_0.001',
 'config_msr_action3d_temporal_1024_16_32_32_0.001',
 'config_msr_action3d_temporal_1024_16_32_64_0.001',
 'config_msr_action3d_temporal_1024_16_32_64_0.025',
 'config_msr_action3d_temporal_1024_16_64_32_0.001',
 'config_msr_action3d_temporal_1024_16_64_64_0.001',
 'config_msr_action3d_temporal_1024_4_128_32_0.025',
 'config_msr_action3d_temporal_1024_4_128_64_0.001',
 'config_msr_action3d_temporal_1024_4_32_32_0.001',
 'config_msr_action3d_temporal_1024_4_32_32_0.025',
 'config_msr_action3d_temporal_1024_4_32_64_0.025',
 'config_msr_action3d_temporal_1024_4_64_128_0.001',
 'config_msr_action3d_temporal_1024_4_64_64_0.001',
 'config_msr_action3d_temporal_2048_16_128_

In [37]:
path = r'log'

model_list_temp = [x for x in os.listdir(path) if 'msr_action3d_temporal' in x]
model_list_temp

['config_msr_action3d_temporal_1024_16_128_128_0.025',
 'config_msr_action3d_temporal_1024_16_128_32_0.001',
 'config_msr_action3d_temporal_1024_16_128_64_0.001',
 'config_msr_action3d_temporal_1024_16_128_64_0.025',
 'config_msr_action3d_temporal_1024_16_32_128_0.001',
 'config_msr_action3d_temporal_1024_16_32_32_0.001',
 'config_msr_action3d_temporal_1024_16_32_64_0.001',
 'config_msr_action3d_temporal_1024_16_32_64_0.025',
 'config_msr_action3d_temporal_1024_16_64_32_0.001',
 'config_msr_action3d_temporal_1024_16_64_64_0.001',
 'config_msr_action3d_temporal_1024_4_128_32_0.025',
 'config_msr_action3d_temporal_1024_4_128_64_0.001',
 'config_msr_action3d_temporal_1024_4_32_32_0.001',
 'config_msr_action3d_temporal_1024_4_32_32_0.025',
 'config_msr_action3d_temporal_1024_4_32_64_0.025',
 'config_msr_action3d_temporal_1024_4_64_128_0.001',
 'config_msr_action3d_temporal_1024_4_64_64_0.001',
 'config_msr_action3d_temporal_2048_16_128_128_0.001',
 'config_msr_action3d_temporal_2048_16_128

In [38]:
for model in model_list_temp:
    if model not in keep_models:
        print(model)
        #shutil.rmtree(os.path.join(path, model))